<a href="https://colab.research.google.com/github/JordMuff2710/PRA3024_AnalysisBigDataPhysics/blob/master/GW.FinalVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Gravitational Waveform: 
In this project we want to create a code that gave the possibility to form the Fisher Matrix of gravitational wave data. In the file below you can find a version (incomplete) of the code. There were several obsticles where the solution(s) to them are still not found. But the code give a general good view of how the code should look like.


Throughout the code there are several comments (and some question marks) to necessary steps in the forming of the Fisher Matrix. 

Eventually with the found results, the necessary plots and diagrams can be formed, because of leck of time this is unfortunately not part of this code. 

This code was also made by external help from Xavier Spronken & Diyon Wickremeratne. 

In [0]:
#Fourier transform of chirp at 3.5PN order#
#This notebook is a cleaned up version of the Fisher matrix calculations.
from sympy import * 
from numpy import *
from math import *
from scipy import *
from matplotlib import *

#some necessary constant throughout the calculations of the Fisher Matrix
G = 6.67*10**(-11) #Gravitational constant 
c = 2.998*10**(8) #speed of light

#They have no effect on the calculation of the Fisher Matrix 
Tc = 1
Phic = 1

# the masses of the black holes in standard units (kg)
M1= 19.891 *10**30 
M2= 19.891 *10**30

TotalMass = (M1+M2)

MassRatio = ((M1*M2)/TotalMass)

ChirpMass = (MassRatio)**(3/5)*TotalMass
#####################################################################


Inertia = 1 #it could be that these should have an actual function?#
Omega = 1 #it could be that these should have an actual function?#

spinSymm = (c*Inertia*Omega)/(G*MassRatio**2)

spinAsymm = -(c*Inertia*Omega)/(G*MassRatio**2)


#measure of how overall strong the gravitational wave is compared to the noise#
SNR= 10 
PN = 2.5

increment = 0.001

#For finding the Fisher Matrix, we need to insert the specific parameters. That is way we insert an empty list where the necessary 
#variables will be defined during the code

Variables = [Tc, Phic, ChirpMass, MassRatio] 
Variables_ET = [Tc, Phic, ChirpMass, MassRatio, spinSymm, spinAsymm] 

In [0]:
# Specifically downloading the datafiles in Google Colab so we can access here and make the necessary datafiles #
!git clone https://github.com/JordMuff2710/PRA3024_AnalysisBigDataPhysics.git

In [0]:
# Chosing the detector: 

WhichDetector = 1

In [0]:
# The PSD of the Advanced LIGO is read in and stored #
# Here we make use of np.interp function; this function is used for curve fitting.#

ValuesReadIn = open("/content/PRA3024_AnalysisBigDataPhysics/GW/PSDAdvLIGO") 

Values = ValuesReadIn.readlines()
ValuesReadIn.close()

FrequencyAdvLIGO=[]
for x in Values:
  FrequencyAdvLIGO.append(float(x.split(' ')[0]))

SensitivityAdvLIGO=[]
for x in Values:
  SensitivityAdvLIGO.append(float(x.split(' ')[6]))

def PSD_AdvLIGO(f):
  f_0 = 215
  f_s = 20
  S0 = 10**(-49)
  x = f/f_0
  if (f >= f_s) :
    PSD_AdvLIGO = S0(x**(-4.14)-5*x**(-2)+(111*(1-x**(2)+(x**(4)/2))/(1+(x**2)/2)))
    return PSD_AdvLIGO
  else:
    r = 1000000
    return r

# The PSD of the Advanced Virgo is read in and stored #
ValuesReadIn = open("/content/PRA3024_AnalysisBigDataPhysics/GW/PSDAdvVirgo") 

Values = ValuesReadIn.readlines()
ValuesReadIn.close()

FrequencyAdvVirgo=[]
for x in Values:
  FrequencyAdvVirgo.append(float(x.split(' ')[0]))

SensitivityAdvVirgo=[]
for x in Values:
  SensitivityAdvVirgo.append(float(x.split(' ')[7]))

def PSD_AdvVirgo(f):
  f_0 = 150
  f_s = 40
  S0 = 9*10**(-46)
  x = f/f_0
  if (f >= f_s):
    PSD_AdvVirgo = S0((6.23*x**(-5))+2*x**(-1)+1+x**2)
    return PSD_AdvVirgo
  else:
    r = 1000000
    return r

ValuesReadIn = open("/content/PRA3024_AnalysisBigDataPhysics/GW/PSDET") 

Values = ValuesReadIn.readlines()
ValuesReadIn.close()

FrequencyET=[]
for x in Values:
  FrequencyET.append(float(x.split('   ')[1]))

SensitivityET=[]
for x in Values:
  SensitivityET.append(float(x.split('   ')[4]))

#def PSD_ET(f): ? 
# Here we should also find an expression that shows the way the PSD is expressed, with corresponding f_s and f_0

In [0]:
#Now with the different datasets availeble, we need to decided which PSD we are going to use#
def PSD(f):
  if WhichDetector == 1:
    outcome = PSD_AdvLIGO(f)
  if WhichDetector == 2:
    outcome = PSD_AdvVirgo(f)
  if WhichDetector == 3:
    outcome = PSD_ET(f)
    return outcome

In [0]:
#defining the different functions specific for forming the Corrections and the Fisher Matrix: 

def v(ChirpMass, MassRatio, f):
  v = ((pi*G*(ChirpMass/(MassRatio**(3/5)))*f)/c**3)**(1/3)
  return v

In [0]:
#defining the corrections of the gravitational wave by first defining the different functions for alphas and make a list of them#
#for sufficiency we will only take the first 5 alphas. 
"Advanced LIGO and Advanced Virgo"
Alpha0 = 1.0

Alpha1 = 0.0

def Alpha2(MassRatio):
  Alpha2 = (20/9)*((743/336)+(11/4)*MassRatio)
  return Alpha2

Alpha3 = -16*pi

def Alpha4(MassRatio):
  Alpha4 = 10*((3058673/1016064)+(5429/1008)*MassRatio+(617/144)*MassRatio**2)
  return Alpha4

def Alpha5(MassRatio, v):
  vslo = (1/6)**(1/2)
  Alpha5 = pi*((38645/756)+(38645/252)*log(v(ChirpMass, MassRatio, f)/vslo)-(65/9)*MassRatio*(1+3*log(v(ChirpMass, MassRatio, f)/vslo)))
  return Alpha5

# Creating a list for easy calculations for the Fourier Domain waveform
Alpha = []
Alpha.append(Alpha0)
Alpha.append(Alpha1)
Alpha.append(Alpha2)
Alpha.append(Alpha3)
Alpha.append(Alpha4)

#print(Alpha)

In [0]:
#Alphas and necessary contributions for the data analysis for the ET
"Einstein Telescope"
 
def delta(MassRatio):
    delta = (1-4*MassRatio)**(1/2)
    return delta

def beta(MassRatio,spinSymm,spinAsymm):
    beta = (113/12 - (13/9)*(MassRatio) )*spinSymm + (113/12)*delta(MassRatio)*spinAsymm
    return beta

def sigma(MassRatio,spinSymm,spinAsymm):
    sigma = (MassRatio)*(721/48*(spinSymm**2 -spinAsymm**2) - (247/48)*(spinSymm**2 - spinAsymm**2)) + (1-2*Massratio)*((719/96)*(spinSymm**2 + spinAsymm**2) - (233/96)*(spinSymm**2 + spinAsymm**2)) + delta(Massratio)*( (719/48)*spinSymm*spinAsymm - 233/48*spinSymm*spinAsymm)
    return sigma
    
def gamma(MassRatio,spinSymm, spinAsymm):
    gamma = (732985/2268 - (24260/81)*MassRatio - (340/9)*(2*MassRatio))*spinSymm + delta(MassRatio)*(732985/2268 + (140/9)*Massratio)*spinAsymm
    return gamma
    
Alpha0Spins = 1.0

Alpha1Spins = 0.0


def Alpha2Spins(MassRatio):
    return 20/9 * (743/336 + 11/4 * MassRatio)

def Alpha3Spins(MassRatio,spinSymm,spinAsymm):
    return 4*beta(MassRatio,spinSymm,spinAsymm)-16*Pi

def Alpha4Spins(MassRatio,spinSymm,spinAsymm):
    return 10*( 3058673/1016064 + 5429/1008*(MassRatio) + 617/144*(MassRatio)**2 - sigma(MassRatio,spinSymm,spinAsymm))

def Alpha5Spins(f,ChirpMass,MassRatio,spinSymm,spinAsymm):
    outcome = (Pi*38645/756 - Pi*(MassRatio)*65/9 - gamma(MassRatio,spinSymm,spinAsymm))*(1+3*log(v(ChirpMass, MassRatio, f)))
    return outcome

AlphaSpins = []
AlphaSpins.append(Alpha0Spins)
AlphaSpins.append(Alpha1Spins)
AlphaSpins.append(Alpha2Spins)
AlphaSpins.append(Alpha3Spins)
AlphaSpins.append(Alpha4Spins)
AlphaSpins.append(Alpha5Spins)

In [0]:
if WhichDetector == 1:
  LowFrequency = 20
if WhichDetector == 2:
  LowFrequency = 40
if WhichDetector == 3:
  LowFrequency = 1

####

if WhichDetector == 1:
  HighFrequency = 215
if WhichDetector == 2:
  HighFrequency = 150
if WhichDetector == 3:
  HighFrequency = 1600

def Amplitude(SNR, increment):
  #perform integral using trapezoidal sums
  Storage = 0
  for i in range (int(LowFrequency//increment), int(HighFrequency//increment - 1)):
   f = i*increment
   fi = (f**(-7/6) / (PSD(f)))
   fi2 = ((f+ increment)**(-7/6) / (PSD(f + increment)))
   Storage += increment * (fi + fi2)/2
   
  # divide SNR by integral multiplied by 4
  A = m.sqrt(SNR/(4*accumulator))
  return A

In [0]:
#Calculations for the Fourier domain waveform, but we make a difference between the different waveforms
#For doing this we need to formulate the Psi function since the H formula rely on this
#To make ourselves a bit easier we put in the definition of H_Normal.  
#Now we can formulate the H function. 

if WhichDetector == 1:
  f = FrequencyAdvLIGO
if WhichDetector == 2:
  f = FrequencyAdvVirgo
if WhichDetector == 3:
  f = FrequencyET
  
def H_Normal(Amplitude, f, Variables, PN):
  
  Psi = 2* pi * f * Tc - Phic - (pi/4) +(3/(128*MassRatio*(v(ChirpMass, MassRatio, f)**5)))*Corrections

  PN *= 2.5
  Corrections = 0
  for i in range(PN):
    Corrections += (Alpha[i]*v**i)

  H_Normal = Amplitude(f)*f**(-7/6)*e**(1j*Psi)
  return H_Normal

def H_ET(Amplitude, f, Variables, PN):

  
  Psi = 2* pi * f * Tc - Phic - (pi/4) +(3/(128*MassRatio*(v(ChirpMass, MassRatio, f)**5)))*Corrections_ET

  PN *= 2.5
  Corrections_ET = 0
  for i in range(PN):
    Corrections += (AlphaSpins[i]*v**i)
  

  H_ET = Amplitude(f)*f**(-7/6)*e**(1j*Psi)
  return H_ET



In [0]:
# specific derivative for the H funciton

''' Calculating the partial derivative of h(f) '''
def H_Partial(f, Amplitude, Variables, PN):   
  dx = Variables[PN]/100
  dVariables = Variables.copy()
  dVariables[PN] += dx
  if WhichDetector == 3: 
    dy = H_ET(Amplitude, f, dVariables, PN) - H_normal(Amplitude, f, Variables, PN)
  elif WhichDetector == 1 or WhichdDetector == 2:
    dy = H_Normal(Amplitude, f, dVariables, PN) - H_normal(Amplitude, f, Variables, PN)
  H_Partial = dy/dx
  return H_Partial

#Now we can form the Fisher Matrix and formulate the full matrix. 
#To do so, we need to form a new matrix with the necessary space (4x4)

def generate_matrix(A, Variables, increment):
  if WhichdDetector == 3:
    n = len(Variables_ET)
  elif WhichdDetector == 1 or WhicWhichdDetectorhdetector == 2: 
    n = len(Variables)
    matrix = np.empty([n,n]) 
  
# Start filling the matrix
  for i in range (n):
    for j in range (n):
      Storage = 0
      
      for k in range(int(LowFrequency//increment), int(HighFrequency//increment-1)):
        x1 = k*increment
        part_i1 = h_partial(x1, A, params,i)
        part_j1 = h_partial(x1, A, params,j)
        f1 = ((part_i1 * conj(part_j1))+(conj(part_i1) * part_j1))/PSD(x1)  
        
        x2 = k*increment + increment
        part_i2 = h_partial(x2, A, params,i)
        part_j2 = h_partial(x2, A, params,j)
        f2 = ((part_i2 * conj(part_j2))+(conj(part_i2) * part_j2))/PSD(x2)
        
        accumulator += real(increment * (f1 + f2)/2)
      matrix[i][j] = accumulator*2

print("Fischer matrix : \n",  matrix)

def extract_from_matrix(matrix):
  #invert matrix
  y = np.linalg.inv(matrix) 
  z = []
  
  #fetch the square root of diagonal elements
  for i in range (len(y)):
    z.append(m.sqrt(y[i][i]))
  
  return z

In [0]:
if WhichDetector ==3:
  print("teta : Tc, Phic, ChirpMass, MassRatio: ", Variables)
  A = Amplitude(SNR, increment)
  fischer = generate_matrix(A, teta, 0.1)
  Results = extract_from_matrix(fischer)
  print("Results for PN =", PN, " : " ,Results)
elif WhichDetector == 1 or WhichDetector == 2: 
  print("teta : Tc, Phic, ChirpMass, MassRatio: ", Variables_ET)
  A = Amplitude(SNR, increment)
  fischer = generate_matrix(A, teta, 0.1)
  Results = extract_from_matrix(fischer)
  print("Results for PN =", PN, " : " ,Results) 
